In [1]:
from os import environ
environ.setdefault("HF_HOME", "/data/hf_models/")

'/data/hf_models/'

In [2]:
from a2a.types import AgentCapabilities, AgentCard, AgentSkill
from adp_core.types import AgentMessage
from adp_core.agent import BaseAgent
from adp_core.chain import BaseLLMChain
from adp_core.orchestration import ParallelAgent
from adp_transformers.chain import ChatCausalMultiTurnsChain

/data/agent_design_pattern/src/transformers/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class Agent(BaseAgent):
    chain: BaseLLMChain

    def execute(self, message: AgentMessage, **kwargs) -> AgentMessage:
        self.state = "running"
        message = self.chain.invoke(message, **kwargs)
        message.execution_result = "success"
        message.origin = self.card.name
        self.state = "idle"
        return message

In [4]:
system_prompt = """You are a genius and creative writer.
"""

user_prompt = """{query}"""

max_iteration = 5

def state_callback(state: str):
    print(f"agent state: {state}")
agent1_skill = AgentSkill(
    id='agent1-skill',
    name="agent1 skill",
    description="self-agent1 skill",
    tags=['agent1']
)
agent1_card = AgentCard(
    name="agent1 agent",
    description="self-agent1 agent",
    skills=[agent1_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
agent2_skill = AgentSkill(
    id='agent2-skill',
    name="agent2 skill",
    description="self-agent2 skill",
    tags=['agent2']
)
agent2_card = AgentCard(
    name="agent2 agent",
    description="self-agent2 agent",
    skills=[agent2_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
parallel_skill = AgentSkill(
    id='parallel-skill',
    name="parallel skill",
    description="self-parallel skill",
    tags=['parallel']
)
parallel_card = AgentCard(
    name="parallel agent",
    description="self-parallel agent",
    skills=[parallel_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
chain = ChatCausalMultiTurnsChain(
    model="HuggingFaceTB/SmolLM3-3B",
    system_prompt=system_prompt,
    user_prompt_template=user_prompt,
    device="cuda")
chain2 = ChatCausalMultiTurnsChain(
    model="ibm-granite/granite-4.0-h-1b",
    system_prompt=system_prompt,
    user_prompt_template=user_prompt,
    device="cuda",
    max_new_tokens=16384)
agent1 = Agent(chain=chain, card=agent1_card, state_change_callback=state_callback)
agent2 = Agent(chain=chain, card=agent2_card, state_change_callback=state_callback)
parallel_agent = ParallelAgent(agents=[agent1, agent2], card=parallel_card, state_change_callback=state_callback)

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.03s/it]
The fast path is not available because one of `(selective_state_update, causal_conv1d_fn, causal_conv1d_update)` is None. Falling back to the naive implementation. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d


In [5]:
query = "Write a short poem with Shakespere's style about love"
message = parallel_agent.execute(AgentMessage(query=query), temperature=1.5, max_new_tokens=16384)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

agent state: parallel agent:running/((agent1 agent:idle)|(agent2 agent:idle))
agent state: parallel agent:running/((agent1 agent:running)|(agent2 agent:idle))
agent state: parallel agent:running/((agent1 agent:running)|(agent2 agent:running))
agent state: parallel agent:running/((agent1 agent:running)|(agent2 agent:idle))
agent state: parallel agent:running/((agent1 agent:idle)|(agent2 agent:idle))
agent state: parallel agent:idle/((agent1 agent:idle)|(agent2 agent:idle))
query='' query_media=None query_media_type=None origin='parallel agent' responses=[('agent2 agent', "\n\n**Sonnet.**\n\nAlack, the might that passion’s fire doth win,  \nA gift of life so sure, yet so unbound!  \nHow many hearts doth seize a fair desire,  \nTill want with fear doth wound it and confound—  \n\nIn mine, true peace has ne’er beheld’s soft hand,  \nYet still my spirits urge my deepest part  \nTo yearn for her as blossoms yearn the sun,  \nTo watch her face is triumph that my heart.—\n\nAh! 'Tis as vain as